In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
import gc
import xgboost as xgb

pd.set_option('display.max_columns', 200)

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/Machine Learning Project - 605/XGBoost/train/train.csv', engine='python')
test_df = pd.read_csv('/content/drive/MyDrive/Machine Learning Project - 605/XGBoost/test/test.csv', engine='python')

In [ ]:
import subprocess
print((subprocess.check_output("lscpu", shell=True).strip()).decode())

Architecture:                         x86_64
CPU op-mode(s):                       32-bit, 64-bit
Address sizes:                        46 bits physical, 48 bits virtual
Byte Order:                           Little Endian
CPU(s):                               2
On-line CPU(s) list:                  0,1
Vendor ID:                            GenuineIntel
Model name:                           Intel(R) Xeon(R) CPU @ 2.20GHz
CPU family:                           6
Model:                                79
Thread(s) per core:                   2
Core(s) per socket:                   1
Socket(s):                            1
Stepping:                             0
BogoMIPS:                             4399.99
Flags:                                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 

In [ ]:
ITERATIONS = 700
EARLY_STOP = 10

params = {'tree_method': 'hist', 'max_depth': 8, 'alpha': 0.1,
          'gamma': 0.3, 'subsample': 0.6, 'scale_pos_weight': 1, 'learning_rate': 0.05,
          'silent': 1, 'objective':'binary:logistic', 'eval_metric': 'auc', 'silent':True,
          'verbose_eval': False}

In [ ]:
train_df.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,var_11,var_12,var_13,var_14,var_15,var_16,var_17,var_18,var_19,var_20,var_21,var_22,var_23,var_24,var_25,var_26,var_27,var_28,var_29,var_30,var_31,var_32,var_33,var_34,var_35,var_36,var_37,var_38,var_39,var_40,var_41,var_42,var_43,var_44,var_45,var_46,var_47,var_48,var_49,var_50,var_51,var_52,var_53,var_54,var_55,var_56,var_57,var_58,var_59,var_60,var_61,var_62,var_63,var_64,var_65,var_66,var_67,var_68,var_69,var_70,var_71,var_72,var_73,var_74,var_75,var_76,var_77,var_78,var_79,var_80,var_81,var_82,var_83,var_84,var_85,var_86,var_87,var_88,var_89,var_90,var_91,var_92,var_93,var_94,var_95,var_96,var_97,...,var_100,var_101,var_102,var_103,var_104,var_105,var_106,var_107,var_108,var_109,var_110,var_111,var_112,var_113,var_114,var_115,var_116,var_117,var_118,var_119,var_120,var_121,var_122,var_123,var_124,var_125,var_126,var_127,var_128,var_129,var_130,var_131,var_132,var_133,var_134,var_135,var_136,var_137,var_138,var_139,var_140,var_141,var_142,var_143,var_144,var_145,var_146,var_147,var_148,var_149,var_150,var_151,var_152,var_153,var_154,var_155,var_156,var_157,var_158,var_159,var_160,var_161,var_162,var_163,var_164,var_165,var_166,var_167,var_168,var_169,var_170,var_171,var_172,var_173,var_174,var_175,var_176,var_177,var_178,var_179,var_180,var_181,var_182,var_183,var_184,var_185,var_186,var_187,var_188,var_189,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,-4.9200,5.7470,2.9252,3.1821,14.0137,0.5745,8.7989,14.5691,5.7487,-7.2393,4.2840,30.7133,10.5350,16.2191,2.5791,2.4716,14.3831,13.4325,-5.1488,-0.4073,4.9306,5.9965,-0.3085,12.9041,-3.8766,16.8911,11.1920,10.5785,0.6764,7.8871,4.6667,3.8743,-5.2387,7.3746,11.5767,12.0446,11.6418,-7.0170,5.9226,-14.2136,16.0283,5.3253,12.9194,29.0460,-0.6940,5.1736,-0.7474,14.8322,11.2668,5.3822,2.0183,10.1166,16.1828,4.9590,2.0771,-0.2154,8.6748,9.5319,5.8056,22.4321,5.0109,-4.7010,21.6374,0.5663,5.1999,8.8600,43.1127,18.3816,-2.3440,23.4104,6.5199,12.1983,13.6468,13.8372,1.3675,2.9423,-4.5213,21.4669,9.3225,16.4597,7.9984,-1.7069,-21.4494,6.7806,11.0924,9.9913,14.8421,0.1812,8.9642,16.2572,...,9.4763,13.3102,26.5376,1.4403,14.7100,6.0454,9.5426,17.1554,14.1104,24.3627,2.0323,6.7602,3.9141,-0.4851,2.5240,1.5093,2.5516,15.5752,-13.4221,7.2739,16.0094,9.7268,0.8897,0.7754,4.2218,12.0039,13.8571,-0.7338,-1.9245,15.4462,12.8287,0.3587,9.6508,6.5674,5.1726,3.1345,29.4547,31.4045,2.8279,15.6599,8.3307,-5.6011,19.0614,11.2663,8.6989,8.3694,11.5659,-16.4727,4.0288,17.9244,18.5177,10.7800,9.0056,16.6964,10.4838,1.6573,12.1749,-13.1324,17.6054,11.5423,15.4576,5.3133,3.6159,5.0384,6.6760,12.6644,2.7004,-0.6975,9.5981,5.4879,-4.7645,-8.4254,20.8773,3.1531,18.5618,7.7423,-10.1245,13.7241,-3.5189,1.7202,-8.4051,9.0164,3.0657,14.3691,25.8398,5.8764,11.8411,-19.7159,17.5743,0.5857,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,3.1468,8.0851,-0.4032,8.0585,14.0239,8.4135,5.4345,13.7003,13.8275,-15.5849,7.8000,28.5708,3.4287,2.7407,8.5524,3.3716,6.9779,13.8910,-11.7684,-2.5586,5.0464,0.5481,-9.2987,7.8755,1.2859,19.3710,11.3702,0.7399,2.7995,5.8434,10.8160,3.6783,-11.1147,1.8730,9.8775,11.7842,1.2444,-47.3797,7.3718,0.1948,34.4014,25.7037,11.8343,13.2256,-4.1083,6.6885,-8.0946,18.5995,19.3219,7.0118,1.9210,8.8682,8.0109,-7.2417,1.7944,-1.3147,8.1042,1.5365,5.4007,7.9344,5.0220,2.2302,40.5632,0.5134,3.1701,20.1068,7.7841,7.0529,3.2709,23.4822,5.5075,13.7814,2.5462,18.1782,0.3683,-4.8210,-5.4850,13.7867,-13.5901,11.0993,7.9022,12.2301,0.4768,6.8852,8.0905,10.9631,11.7569,-1.2722,24.7876,26.6881,...,-13.6950,8.4068,35.4734,1.7093,15.1866,2.6227,7.3412,32.0888,13.9550,13.0858,6.6203,7.1051,5.3523,8.5426,3.6159,4.1569,3.0454,7.8522,-11.5100,7.5109,31.5899,9.5018,8.2736,10.1633,0.1225,12.5942,14.5697,2.4354,0.8194,16.5346,12.4205,

In [ ]:
import time
nfold = 5
skf = StratifiedKFold(n_splits=nfold, shuffle=True, random_state=2019)

oof = np.zeros(len(train_df))
predictions = np.zeros(len(test_df))

target = 'target'
predictors = train_df.columns.values.tolist()[2:]

total_train_time = 0
total_pred_time = 0

i = 1
for train_index, valid_index in skf.split(train_df, train_df.target.values):
    print(f"\nFold {i}")

    xg_train = xgb.DMatrix(train_df.iloc[train_index][predictors].values,
                           train_df.iloc[train_index][target].values)
    xg_valid = xgb.DMatrix(train_df.iloc[valid_index][predictors].values,
                           train_df.iloc[valid_index][target].values)

    # Start timing training
    start_train = time.time()
    clf = xgb.train(params, xg_train, ITERATIONS, evals=[(xg_train, "train"), (xg_valid, "eval")])
    end_train = time.time()
    total_train_time += (end_train - start_train)

    # Start timing prediction
    start_pred = time.time()
    oof[valid_index] = clf.predict(xgb.DMatrix(train_df.iloc[valid_index][predictors].values))
    predictions += clf.predict(xgb.DMatrix(test_df[predictors].values)) / nfold
    end_pred = time.time()
    total_pred_time += (end_pred - start_pred)

    i += 1

print(f"\nTotal training time: {total_train_time:.2f} seconds")
print(f"Total prediction time: {total_pred_time:.2f} seconds")



Fold 1


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [00:03:39] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "silent", "verbose_eval" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	train-auc:0.66578	eval-auc:0.63683
[1]	train-auc:0.70147	eval-auc:0.66735
[2]	train-auc:0.72557	eval-auc:0.68515
[3]	train-auc:0.73544	eval-auc:0.69398
[4]	train-auc:0.74822	eval-auc:0.70308
[5]	train-auc:0.75179	eval-auc:0.70316
[6]	train-auc:0.76415	eval-auc:0.70969
[7]	train-auc:0.77234	eval-auc:0.71536
[8]	train-auc:0.77653	eval-auc:0.71793
[9]	train-auc:0.78184	eval-auc:0.72222
[10]	train-auc:0.78892	eval-auc:0.72624
[11]	train-auc:0.79374	eval-auc:0.72962
[12]	train-auc:0.79990	eval-auc:0.73253
[13]	train-auc:0.80706	eval-auc:0.73787
[14]	train-auc:0.81127	eval-auc:0.74048
[15]	train-auc:0.81548	eval-auc:0.74333
[16]	train-auc:0.82043	eval-auc:0.74589
[17]	train-auc:0.82540	eval-auc:0.74878
[18]	train-auc:0.83263	eval-auc:0.75333
[19]	train-auc:0.83711	eval-auc:0.75531
[20]	train-auc:0.83869	eval-auc:0.75663
[21]	train-auc:0.84302	eval-auc:0.76041
[22]	train-auc:0.84739	eval-auc:0.76403
[23]	train-auc:0.85266	eval-auc:0.76717
[24]	train-auc:0.85695	eval-auc:0.76986
[25]	train

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [00:10:06] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "silent", "verbose_eval" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	train-auc:0.66857	eval-auc:0.63739
[1]	train-auc:0.69340	eval-auc:0.66546
[2]	train-auc:0.71105	eval-auc:0.68098
[3]	train-auc:0.72349	eval-auc:0.68777
[4]	train-auc:0.74097	eval-auc:0.70337
[5]	train-auc:0.75368	eval-auc:0.71355
[6]	train-auc:0.75953	eval-auc:0.71527
[7]	train-auc:0.76770	eval-auc:0.72113
[8]	train-auc:0.77472	eval-auc:0.72643
[9]	train-auc:0.78370	eval-auc:0.73089
[10]	train-auc:0.79321	eval-auc:0.73794
[11]	train-auc:0.79275	eval-auc:0.73740
[12]	train-auc:0.79958	eval-auc:0.74151
[13]	train-auc:0.80740	eval-auc:0.74703
[14]	train-auc:0.81311	eval-auc:0.75052
[15]	train-auc:0.81951	eval-auc:0.75436
[16]	train-auc:0.82589	eval-auc:0.75974
[17]	train-auc:0.82988	eval-auc:0.76219
[18]	train-auc:0.83463	eval-auc:0.76499
[19]	train-auc:0.83757	eval-auc:0.76544
[20]	train-auc:0.84097	eval-auc:0.76723
[21]	train-auc:0.84471	eval-auc:0.77012
[22]	train-auc:0.84881	eval-auc:0.77216
[23]	train-auc:0.85327	eval-auc:0.77461
[24]	train-auc:0.85770	eval-auc:0.77766
[25]	train

...

[0]	train-auc:0.67039	eval-auc:0.64325
[1]	train-auc:0.69914	eval-auc:0.66669
[2]	train-auc:0.71636	eval-auc:0.68055
[3]	train-auc:0.73965	eval-auc:0.69507
[4]	train-auc:0.75298	eval-auc:0.70506
[5]	train-auc:0.76026	eval-auc:0.71047
[6]	train-auc:0.76685	eval-auc:0.71416
[7]	train-auc:0.77482	eval-auc:0.71870
[8]	train-auc:0.77964	eval-auc:0.72315
[9]	train-auc:0.78492	eval-auc:0.72676
[10]	train-auc:0.79060	eval-auc:0.72972
[11]	train-auc:0.79635	eval-auc:0.73343
[12]	train-auc:0.80169	eval-auc:0.73811
[13]	train-auc:0.80515	eval-auc:0.74000
[14]	train-auc:0.80969	eval-auc:0.74295
[15]	train-auc:0.81561	eval-auc:0.74651
[16]	train-auc:0.81830	eval-auc:0.74797
[17]	train-auc:0.82385	eval-auc:0.75194
[18]	train-auc:0.82949	eval-auc:0.75547
[19]	train-auc:0.83347	eval-auc:0.75848
[20]	train-auc:0.83741	eval-auc:0.76093
[21]	train-auc:0.84151	eval-auc:0.76373
[22]	train-auc:0.84640	eval-auc:0.76765
[23]	train-auc:0.85102	eval-auc:0.77117
[24]	train-auc:0.85528	eval-auc:0.77348
[25]	train

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [00:22:54] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "silent", "verbose_eval" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	train-auc:0.66774	eval-auc:0.64581
[1]	train-auc:0.70537	eval-auc:0.67480
[2]	train-auc:0.72758	eval-auc:0.69223
[3]	train-auc:0.74281	eval-auc:0.70140
[4]	train-auc:0.75167	eval-auc:0.70700
[5]	train-auc:0.75882	eval-auc:0.71300
[6]	train-auc:0.76556	eval-auc:0.71779
[7]	train-auc:0.77121	eval-auc:0.72262
[8]	train-auc:0.77431	eval-auc:0.72676
[9]	train-auc:0.78377	eval-auc:0.73161
[10]	train-auc:0.78800	eval-auc:0.73450
[11]	train-auc:0.79513	eval-auc:0.73876
[12]	train-auc:0.80059	eval-auc:0.74194
[13]	train-auc:0.80814	eval-auc:0.74614
[14]	train-auc:0.81338	eval-auc:0.74900
[15]	train-auc:0.82093	eval-auc:0.75398
[16]	train-auc:0.82610	eval-auc:0.75707
[17]	train-auc:0.83152	eval-auc:0.76070
[18]	train-auc:0.83479	eval-auc:0.76330
[19]	train-auc:0.83807	eval-auc:0.76512
[20]	train-auc:0.84166	eval-auc:0.76713
[21]	train-auc:0.84542	eval-auc:0.77024
[22]	train-auc:0.85002	eval-auc:0.77276
[23]	train-auc:0.85217	eval-auc:0.77440
[24]	train-auc:0.85521	eval-auc:0.77536
[25]	train

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [00:29:14] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "silent", "verbose_eval" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	train-auc:0.66350	eval-auc:0.63807
[1]	train-auc:0.70366	eval-auc:0.67211
[2]	train-auc:0.72040	eval-auc:0.68655
[3]	train-auc:0.73610	eval-auc:0.69598
[4]	train-auc:0.74754	eval-auc:0.70230
[5]	train-auc:0.75669	eval-auc:0.70698
[6]	train-auc:0.76738	eval-auc:0.71638
[7]	train-auc:0.77392	eval-auc:0.72035
[8]	train-auc:0.78093	eval-auc:0.72440
[9]	train-auc:0.78874	eval-auc:0.72845
[10]	train-auc:0.79594	eval-auc:0.73310
[11]	train-auc:0.80319	eval-auc:0.73586
[12]	train-auc:0.80787	eval-auc:0.73824
[13]	train-auc:0.81248	eval-auc:0.74278
[14]	train-auc:0.81863	eval-auc:0.74625
[15]	train-auc:0.82408	eval-auc:0.74871
[16]	train-auc:0.82882	eval-auc:0.75218
[17]	train-auc:0.83188	eval-auc:0.75515
[18]	train-auc:0.83731	eval-auc:0.75993
[19]	train-auc:0.84297	eval-auc:0.76230
[20]	train-auc:0.84706	eval-auc:0.76491
[21]	train-auc:0.84974	eval-auc:0.76692
[22]	train-auc:0.85427	eval-auc:0.76972
[23]	train-auc:0.85638	eval-auc:0.77105
[24]	train-auc:0.85936	eval-auc:0.77419
[25]	train

In [ ]:
import psutil, os
def print_memory_usage_gb():
  # CPU RAM usage
  process = psutil.Process(os.getpid())
  ram_usage_gb = process.memory_info().rss / (1024 ** 3)  # <<< divide by 1024³
  print(f"CPU RAM Usage: {ram_usage_gb:.4f} GB")

print_memory_usage_gb()

CPU RAM Usage: 3.9801 GB
